In [1]:
"""
    Python implementation of POW
    Credit: https://github.com/mycoralhealth/blockchain-tutorial
"""
from datetime import datetime
import time
from hashlib import sha256
import json, requests
from random import randint

In [2]:
if not "":
    print("Asd")

Asd


In [3]:
class Blockchain(object):
    

    def __init__(self, _genesisBlock, account):
        """
            If the genesis block is valid, create chain
        """
        self.blockChain = []
        self.tempBlocks = []
        #self.candidateBlocks = [] #constains block
        self.myCurrBlock = {}
        #self.announcements = []
        self.validators = set() # stakers and balance
        #self.unconfirmed_txns = []
        self.nodes = set()
        self.myAccount = {'Address': '', 'Weight': 0, 'Age': 0}
        self.myAccount['Address'] = account['Address']
        self.myAccount['Weight'] = account['Weight']
        try:
            genesisBlock = self.generate_genesis_block(_genesisBlock)
            if self.is_block_valid(genesisBlock):
                self.blockChain.append(genesisBlock)
            else:
                raise Exception('Unable to verify block')
        except Exception as e:
            print('Invalid genesis block.\nOR\n' + str(e))
            
    ## 블록이 잘 만들어 진건지 보는건데!!
    def is_block_valid(self, block, prevBlock={}):
        try:
            #우선 만들러는 블록에서 해시를 빼고
            _hash = block.pop('Hash')
        except KeyError as e:
            ## 해시가 없으면 당연히 invalid 한 블럭 > False 리턴
            return False
        try:
            ## 해시 뺸걸로 다시 해시함스 해서 HASH2 만든다음 비교
            hash2 = self.hasher(block)
            assert _hash == hash2
        except AssertionError as e:
            ## 당연히 같아야하는데 다르면 nvalid 한 블럭 > False 리턴
            return False
        
        ## PREVBLOCK 이 있었으면 그것의 해시, 아니면 '' 로 해서  
        prevHash = prevBlock['Hash'] if prevBlock else ''
        block['Hash'] = _hash
        
        # 블록체인 값이 있으면!!
        if self.blockChain:
            # prevHash 가 "" (첫 블록) 이면 기존 블록의 마지막 해시를, 아니면 위에서 구한 prevHash로
            prevHash = self.blockChain[-1]['Hash'] if not prevHash else prevHash
            try:
                assert prevHash == block["PrevHash"]
            except AssertionError as e:
                if prevHash == self.blockChain[0]['Hash']:
                    block['Hash'] = _hash
                    return True
                block['Hash'] = _hash
                return False
        block['Hash'] = _hash
        return True

    def generate_new_block(self, bpm=randint(53, 63), oldBlock='', address=''):
        if self.myCurrBlock:
            return self.myCurrBlock
        prevHash = self.blockChain[-1]['Hash']
        
        ## 새로 만들 블록의 index 정하기
        ## oldBlock 에 빈칸이면 현재 블록의 길이가 인덱스로 (0부터 시작이니)
        # 원가 있으면 올드블록인덱스에 1 더해라
        index = len(self.blockChain) if not oldBlock else oldBlock['Index'] + 1
        
        ## 새로 블록에 내 노드 적어두기!!!
        ## 어드래스에 빈칸이면 지금 내 어카운트 어드레스로
        address =  self.get_validator(self.myAccount) if not address else address
        newBlock = {
            "Index": index,
            "Timestamp": str(datetime.now()),
            "BPM": bpm, #instead of transactions
            "PrevHash": prevHash,
            "Validator": address
        }
        newBlock["Hash"] = self.hasher(newBlock)
        assert self.is_block_valid(newBlock)
        self.myCurrBlock = newBlock
        return newBlock

    def get_blocks_from_nodes(self):
        if self.nodes:
            for node in self.nodes:
                #resp = requests.get('http://{}/newblock'.format(node))
                node.add_another_block(self.myCurrBlock)
                resp = node.generate_new_block()
                if self.is_block_valid(resp): #resp.json()
                    #self.tempBlocks.append(resp.json())
                    if not resp['Validator'] in self.validators:
                        self.tempBlocks.append(resp)
                        self.validators.add(resp['Validator'])

    def add_another_block(self, another_block):
        if self.is_block_valid(another_block):
            if not another_block['Validator'] in self.validators:
                self.tempBlocks.append(another_block)
                self.validators.add(another_block['Validator'])


    def pick_winner(self):
        ### 누가누가 블록 만들래!! 만들사람 뽑기
        """Creates a lottery pool of validators and choose the validator
            who gets to forge the next block. Random selection weighted by amount of token staked
            Do this every 30 seconds
        """
        winner = []

        self.tempBlocks.append(self.myCurrBlock)
        self.validators.add(self.myCurrBlock['Validator'])
        for validator in self.validators:
            acct = (validator.rsplit(sep=', '))
            acct.append(int(acct[1]) * int(acct[2]))
            if winner and acct[-1]:
                winner = acct if winner[-1] < acct[-1] else winner
            else:
                winner = acct if acct[-1] else winner
        if winner:
            return winner
        for validator in self.validators:
            acct = (validator.rsplit(sep=', '))
            acct.append((int(acct[1]) + int(acct[2]))/len(acct[0]))
            if winner:
                winner = acct if winner[-1] < acct[-1] else winner
            else:
                winner = acct
        return winner

    def pos(self):
        """
        #get other's stakes
        #add owns claim
        #pick winner
        """

        print(str(self.myAccount) + ' =======================> Getting Valid chain\n')
        self.resolve_conflict()
        time.sleep(1)
        self._pos()
        print('***Calling other nodes to announce theirs***' + "\n")
        time.sleep(1)
        for node in self.nodes:
            node._pos()
        time.sleep(1)
        for block in self.tempBlocks:
            validator = block['Validator'].rsplit(', ')
            if validator[0] == self.pick_winner()[0]:
                new_block = block
                break
            else:
                pass
        print('New Block ====> ' + str(new_block) + "\n")
        time.sleep(1)
        self.add_new_block(new_block)
        for node in self.nodes:
            node.add_new_block(new_block)
        print('Process ends' + "\n")
        
    def announce_winner(self):
        self.blockChain.append(self.myCurrBlock)

    def add_new_block(self, block):
        if self.is_block_valid(block):
            #check index too
            self.blockChain.append(block)
            acct = block['Validator'].rsplit(', ')
            if self.myAccount['Address'] != acct[0]:
                self.myAccount['Age'] += 1
            else:
                self.myAccount['Weight'] += (randint(1, 10) * self.myAccount['Age'])
                self.myAccount['Age'] = 0
        self.tempBlocks = []
        self.myCurrBlock = {}
        self.validators = set()



    def _pos(self):
        print("Coming from ==========================> " + str(self.myAccount) + "\n")
        time.sleep(1)
        print('***Generating new stake block***' + "\n")
        time.sleep(1)
        self.generate_new_block()
        print('***Exchanging temporary blocks with other nodes***' + "\n")
        time.sleep(1)
        self.get_blocks_from_nodes()
        print('***Picking a winner***' + "\n")
        time.sleep(1)
        print("Winner is =======================> " + str(self.pick_winner()) + "\n")
    
    def resolve_conflict(self):
        for node in self.nodes:
            if len(node.blockChain) > len(self.blockChain):
                if self.is_chain_valid(node.blockChain):
                    print('***Replacing node***' + "\n")
                    self.blockChain = node.blockChain
                    return
        print('***My chain is authoritative***' + "\n")
        return

    def is_chain_valid(self, chain):
        _prevBlock = ''
        for block in chain:
            if self.is_block_valid(block, prevBlock=_prevBlock):
                _prevBlock = block
            else:
                return False
        return True


    def add_new_node(self, new_node):
        self.nodes.add(new_node)
        new_node.add_another_node(self)

    def add_another_node(self, another_node):
        self.nodes.add(another_node)

    @staticmethod
    def hasher(block):
        block_string = json.dumps(block, sort_keys=True).encode()
        return sha256(block_string).hexdigest()

    @staticmethod
    def get_validator(address):
        return ', '.join([address['Address'], str(address['Weight']), str(address['Age'])])

    def generate_genesis_block(self, genesisblock):
        address = {'Address': 'genesisblock_address', 'Weight': 50, 'Age': 0}
        address = self.get_validator(address)
        genesisblock['Index'] = 0 if not genesisblock['Index'] else genesisblock['Index']
        genesisblock['Timestamp'] = str(datetime.now()) if not genesisblock['Timestamp'] else genesisblock['Timestamp']
        genesisblock['BPM'] = 0 if not genesisblock['BPM'] else genesisblock['BPM']
        genesisblock['PrevHash'] = '0000000000000000'
        genesisblock['Validator'] = address if not genesisblock['Validator'] else genesisblock['Validator']
        genesisblock['Hash'] = self.hasher(genesisblock)
        return genesisblock       

In [4]:
DATE = datetime.now()
GENESIS_BLOCK = {
    "Index": 0,
    "Timestamp": str(DATE),
    "BPM": 0, #instead of transactions
    "PrevHash": "",
    "Validator": "" #address to receive the reward {validator, weight, age}
}

In [5]:
account = {'Address': 'drfirst', 'Weight': 50}
blockchain = Blockchain(GENESIS_BLOCK, account)
blockchain.generate_new_block(52)

{'Index': 1,
 'Timestamp': '2022-07-12 09:42:21.015448',
 'BPM': 52,
 'PrevHash': 'f60360433a3739083e58dc16891c21c347d9cdf32d6e229b4c730667ed7453d5',
 'Validator': 'drfirst, 50, 0',
 'Hash': 'c0623758c58bdb40cb4062166eed1143cad9371f7fcbfe2c57a0889d879fa6f8'}

In [6]:
blockchain.blockChain

[{'Index': 0,
  'Timestamp': '2022-07-12 09:42:14.226118',
  'BPM': 0,
  'PrevHash': '0000000000000000',
  'Validator': 'genesisblock_address, 50, 0',
  'Hash': 'f60360433a3739083e58dc16891c21c347d9cdf32d6e229b4c730667ed7453d5'}]

In [4]:
    account = {'Address': 'drfirst', 'Weight': 50}
    blockchain = Blockchain(GENESIS_BLOCK, account)
    blockchain.generate_new_block(52)

    clients = [blockchain]
    
    blockchain.pick_winner()
    #check if temp blocks are same

    blockchain.pos()

{'Address': 'drfirst', 'Weight': 50, 'Age': 0} =======================> Getting Valid chain

***My chain is authoritative***

Coming from ==========================> {'Address': 'drfirst', 'Weight': 50, 'Age': 0}

***Generating new stake block***

***Exchanging temporary blocks with other nodes***

***Picking a winner***

Winner is =======================> ['drfirst', '50', '0', 7.142857142857143]

***Calling other nodes to announce theirs***

New Block ====> {'Index': 1, 'Timestamp': '2022-07-12 09:04:56.704048', 'BPM': 52, 'PrevHash': '99b7bd5c325b689fa4023dd35f7267361f2dacef0e805a823c8ea9557670e353', 'Validator': 'drfirst, 50, 0', 'Hash': '4f16961b29413d526b505e6fbd79410741245adf93d3c55dfc62c1d3e0b28c53'}

Process ends



In [6]:
while True:
    print('============================================ \n\n')
    client = clients[0]
    client.pos()



{'Address': 'eltneg', 'Weight': 50, 'Age': 0} =======================> Getting Valid chain

***My chain is authoritative***

Coming from ==========================> {'Address': 'eltneg', 'Weight': 50, 'Age': 0}

***Generating new stake block***

***Exchanging temporary blocks with other nodes***

***Picking a winner***

Winner is =======================> ['eltneg', '50', '0', 8.333333333333334]

***Calling other nodes to announce theirs***

New Block ====> {'Index': 2, 'Timestamp': '2022-07-11 21:14:49.219647', 'BPM': 56, 'PrevHash': '3ef1c2dadbcdc5d103392c7e9d9d42e6105f26b3ecb86e9d62e6d3703829ff02', 'Validator': 'eltneg, 50, 0', 'Hash': '40a952e24b2cd8ce40d897606186d2ffcef3aca394cf6e95f63c7ba62a64253b'}

Process ends



{'Address': 'eltneg', 'Weight': 50, 'Age': 0} =======================> Getting Valid chain

***My chain is authoritative***

Coming from ==========================> {'Address': 'eltneg', 'Weight': 50, 'Age': 0}

***Generating new stake block***

***Exchanging tempora

KeyboardInterrupt: 

In [8]:
blockchain.blockChain 

[{'Index': 0,
  'Timestamp': '2022-07-11 21:13:46.042558',
  'BPM': 0,
  'PrevHash': '0000000000000000',
  'Validator': 'eltneg, 50, 0',
  'Hash': '22948b46b04b5a53a8a8ff16523b11decf35ea91973b7681bf3c43a7e753c0a6'},
 {'Index': 1,
  'Timestamp': '2022-07-11 21:14:26.864952',
  'BPM': 52,
  'PrevHash': '22948b46b04b5a53a8a8ff16523b11decf35ea91973b7681bf3c43a7e753c0a6',
  'Validator': 'eltneg, 50, 0',
  'Hash': '3ef1c2dadbcdc5d103392c7e9d9d42e6105f26b3ecb86e9d62e6d3703829ff02'},
 {'Index': 2,
  'Timestamp': '2022-07-11 21:14:49.219647',
  'BPM': 56,
  'PrevHash': '3ef1c2dadbcdc5d103392c7e9d9d42e6105f26b3ecb86e9d62e6d3703829ff02',
  'Validator': 'eltneg, 50, 0',
  'Hash': '40a952e24b2cd8ce40d897606186d2ffcef3aca394cf6e95f63c7ba62a64253b'},
 {'Index': 3,
  'Timestamp': '2022-07-11 21:14:57.227628',
  'BPM': 56,
  'PrevHash': '40a952e24b2cd8ce40d897606186d2ffcef3aca394cf6e95f63c7ba62a64253b',
  'Validator': 'eltneg, 50, 0',
  'Hash': 'bfdfeb0cdf36250f3ea9e7fe23c078c1afc958409780226f11d52ab03